## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote
from sklearn.preprocessing import MinMaxScaler

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
import statistics
from sklearn.metrics import classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'home'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['TAG-Barn [kW]',
 'TAG-Dishwasher [kW]',
 'TAG-Fridge [kW]',
 'TAG-Furnace 1 [kW]',
 'TAG-Furnace 2 [kW]',
 'TAG-Garage door [kW]',
 'TAG-Home office [kW]',
 'TAG-House overall [kW]',
 'TAG-Kitchen 12 [kW]',
 'TAG-Kitchen 14 [kW]',
 'TAG-Kitchen 38 [kW]',
 'TAG-Living room [kW]',
 'TAG-Microwave [kW]',
 'TAG-Solar [kW]',
 'TAG-Well [kW]',
 'TAG-Wine cellar [kW]',
 'TAG-apparentTemperature',
 'TAG-dewPoint',
 'TAG-gen [kW]',
 'TAG-humidity',
 'TAG-precipIntensity',
 'TAG-precipProbability',
 'TAG-pressure',
 'TAG-temperature',
 'TAG-use [kW]',
 'TAG-visibility',
 'TAG-windBearing',
 'TAG-windSpeed']

## TAG Name format 변환 

* 위의 과정에서 Smart home dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

* TAG-windBearing, TAG-windSpeed Tag Name 사용하여 예제 진행 

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = name[-2:]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

'TAG-windBearing','TAG-windSpeed'
2


## Smart Home Dataset 로드

* 데이터 로드시 train, validation, test 데이터 셋을 각각 Load

* 예제는 각 데이터 셋 당 1시간의 데이터를 사용하는 것으로 설정

In [6]:
# 데이터 로드 파라미터 설정

# tag table 이름 설정
table = 'home'
# tag name 설정
name = quote(tags_, safe=":/")
# 시간 포멧 설정 
timeformat = 'Default'

# Train , validation , test 데이터 셋 설정
# 학습 데이터 시작 시간 설정
start_time_train = '2016-01-01 14:00:00'
# 학습 데이터  끝 시간 설정
end_time_train = '2016-01-01 15:00:00'

# 검증 데이터 시작 시간 설정
start_time_val = '2016-01-01 15:00:00'
# 검증 데이터 끝 시간 설정
end_time_val = '2016-01-01 16:00:00'

# 테스트 데이터 시작 시간 설정
start_time_test = '2016-01-01 16:00:00'
# 테스트 데이터 끝 시간 설정 
end_time_test = '2016-01-01 17:00:00'

In [7]:
# 데이터 로드 함수
# '1D': 일간 간격 (1일)
# '1H': 시간 간격 (1시간)
# '1T' 또는 'min': 분 간격 (1분)
# '1S': 초 간격 (1초)
def data_load(table, name, start_time, end_time, timeformat, resample_time):
    
    # URL 인코딩
    start_time = quote(start_time)
    end_time = quote(end_time)
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # time index 설정
    df = df.set_index(pd.to_datetime(df['TIME']))
    df = df.drop(['TIME'], axis=1)
    
    # 1초간격 resampling
    # 원하는 간격으로 수정 가능 일, 시, 분 등등 
    df = df.resample(f'{resample_time}').mean()
    
    return df

In [8]:
# 학습 데이터 로드
train = data_load(table, name, start_time_train, end_time_train, timeformat, "1s")
# 검증 데이터 로드
valid = data_load(table, name, start_time_val, end_time_val, timeformat, "1s")
# 학습 데이터 로드
test = data_load(table, name, start_time_test, end_time_test, timeformat, "1s")

In [9]:
print(train)
print(valid)
print(test)

NAME                 TAG-windBearing  TAG-windSpeed
TIME                                               
2016-01-01 14:00:00            282.0           9.18
2016-01-01 14:00:01            282.0           9.18
2016-01-01 14:00:02            282.0           9.18
2016-01-01 14:00:03            282.0           9.18
2016-01-01 14:00:04            282.0           9.18
...                              ...            ...
2016-01-01 14:59:56            253.0          11.30
2016-01-01 14:59:57            253.0          11.30
2016-01-01 14:59:58            253.0          11.30
2016-01-01 14:59:59            253.0          11.30
2016-01-01 15:00:00            253.0          11.30

[3601 rows x 2 columns]
NAME                 TAG-windBearing  TAG-windSpeed
TIME                                               
2016-01-01 15:00:00            253.0          11.30
2016-01-01 15:00:01            253.0          11.30
2016-01-01 15:00:02            253.0          11.30
2016-01-01 15:00:03            253.0   

## 데이터 전처리

* 각 데이터별로 MinMax Scaling 적용

In [10]:
# 스케일러 설정
scaler = MinMaxScaler()

# 스케일러 적용
train_ = scaler.fit_transform(train.values)
valid_ = scaler.transform(valid.values)
test_ = scaler.transform(test.values)

# 데이터 프레임 설정
train_scaled = pd.DataFrame(train_ , columns=train.columns)
valid_scaled = pd.DataFrame(valid_ , columns=valid.columns)
test_scaled = pd.DataFrame(test_ , columns=test.columns)

# 타임 인덱스 재설정
train_scaled.index = train.index
valid_scaled.index = valid.index
test_scaled.index = test.index

In [11]:
print(train_scaled)
print(valid_scaled)
print(test_scaled)

NAME                 TAG-windBearing  TAG-windSpeed
TIME                                               
2016-01-01 14:00:00         0.966667       0.563459
2016-01-01 14:00:01         0.966667       0.563459
2016-01-01 14:00:02         0.966667       0.563459
2016-01-01 14:00:03         0.966667       0.563459
2016-01-01 14:00:04         0.966667       0.563459
...                              ...            ...
2016-01-01 14:59:56         0.644444       0.859135
2016-01-01 14:59:57         0.644444       0.859135
2016-01-01 14:59:58         0.644444       0.859135
2016-01-01 14:59:59         0.644444       0.859135
2016-01-01 15:00:00         0.644444       0.859135

[3601 rows x 2 columns]
NAME                 TAG-windBearing  TAG-windSpeed
TIME                                               
2016-01-01 15:00:00         0.644444       0.859135
2016-01-01 15:00:01         0.644444       0.859135
2016-01-01 15:00:02         0.644444       0.859135
2016-01-01 15:00:03         0.644444   

## 윈도우 데이터 셋 설정

시계열 데이터 학습을 위해서는 윈도우 사이즈와 슬라이딩 값을 설정해야함

* window size : 몇개의 시간으로 묶을지에 대한 설정
* step size : window 가 이동하는 시간 간격

In [13]:
# 슬라이딩 윈도우 데이터셋 설정 
class SlidingWindowDataset(Dataset):
    def __init__(self, data, window_size, step_size):
        self.data = data
        self.window_size = window_size
        self.step_size = step_size
        self.windows = self._create_windows()
    
    # 슬라이딩 윈도우 설정 
    def _create_windows(self):
        windows = []
        for i in range(0, len(self.data) - self.window_size + 1, self.step_size):
            window = self.data[i:i + self.window_size]
            windows.append(torch.Tensor(window.values))
        return windows
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return self.windows[idx]

In [14]:
# 슬라이딩 윈도우 설정
window_size = 3
step_size = 1 

# 데이터 셋 설정 
train_ = SlidingWindowDataset(train_scaled, window_size, step_size)
valid_ = SlidingWindowDataset(valid_scaled, window_size, step_size)
test_ = SlidingWindowDataset(test_scaled, window_size, step_size)

# 데이터 로더 설정
train_dataloader = DataLoader(train_, batch_size=32, shuffle=False)
valid_dataloader = DataLoader(valid_, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_, batch_size=32, shuffle=False)

In [15]:
print(list(train_dataloader)[0].shape)

torch.Size([32, 3, 2])


## 학습 모델 설정 

* LSTM AE 기본 모델 사용

In [16]:
# LSTM Autoencoder 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # 인코더 LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, 2*hidden_dim)
        
        # 디코더 LSTM
        self.decoder_fc = nn.Linear(2*hidden_dim, hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)

    def forward(self, x):
        # 인코더 부분
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # 디코더 부분
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        return output

In [17]:
# 모델 설정 파라미터

# 입력 데이터 컬럼 수
# Tag Name 수와 동일 
input_dim = len(tags)

# LSMT hidden state 크기
hidden_dim = 2*len(tags)

# layer 수
num_layers = 3

# 학습률 
learning_rate = 0.01

# 모델 초기화
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(2, 4, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=4, out_features=8, bias=True)
  (decoder_fc): Linear(in_features=8, out_features=4, bias=True)
  (decoder_lstm): LSTM(4, 2, num_layers=3, batch_first=True)
)


## 모델 학습 설정

* 학습 중 Loss 값이 제일 낮은 모델을 저장 

In [18]:
train_loss = []
total_step = len(train_dataloader)
epoch_in = trange(100, desc='training')
best_Loss= np.inf

for epoch in epoch_in:
    model.to(device)
    model.train()
    running_loss = 0.0

    preds_ = []
    targets_ = []

    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data.to(device).float()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}')

    
    if best_Loss > np.mean(train_loss):
        best_Loss = np.mean(train_loss)
        torch.save(model, f'./result/Smart_home_LSTM_AE.pt')
        print('모델 저장')
    epoch_in.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")

training:   0%|          | 0/100 [00:00<?, ?it/s]


train loss: 0.11583049455245512
모델 저장

train loss: 0.09801602310426863
모델 저장

train loss: 0.09103955404907134
모델 저장

train loss: 0.08712847792261189
모델 저장

train loss: 0.08458687174916926
모델 저장

train loss: 0.08279100409605894
모델 저장

train loss: 0.08144886465213354
모델 저장

train loss: 0.08040474738489911
모델 저장

train loss: 0.0795677098365008
모델 저장

train loss: 0.0788807465470256
모델 저장

train loss: 0.0783061602357805
모델 저장

train loss: 0.07781798646656198
모델 저장

train loss: 0.07739774099236686
모델 저장

train loss: 0.0770318977666934
모델 저장

train loss: 0.07671032088512011
모델 저장

train loss: 0.07642526288923243
모델 저장

train loss: 0.07617069679367676
모델 저장

train loss: 0.0759418656626687
모델 저장

train loss: 0.07573496423397971
모델 저장

train loss: 0.07554691351014725
모델 저장

train loss: 0.07537519451296698
모델 저장

train loss: 0.07521772793324083
모델 저장

train loss: 0.07507277853541841
모델 저장

train loss: 0.07493888688159374
모델 저장

train loss: 0.07481481213798434
모델 저장

train loss: 0.074699492210519

## 임계값 설정

* validation data를 사용하여 임계값 계산 
1) 평균 + 편차 
2) Max 값
3) 99% - std 값

In [19]:
# 베스트 모델 로드
model_ = torch.load(f'./result/Smart_home_LSTM_AE.pt') 

In [20]:
# validation data 재구성 Loss 계산 
valid_loss = []
with torch.no_grad():
    
    for batch_idx, valid_data in enumerate(valid_dataloader):

        inputs_val = valid_data.to(device).float()

        outputs_val = model_(inputs_val)
        loss = criterion(outputs_val, inputs_val)
        
        valid_loss.append(loss.item())
        
# 임계값 설정
threshold_1 =  statistics.mean(valid_loss) + statistics.stdev(valid_loss)
threshold_2 =  max(valid_loss)
threshold_3 =  np.percentile(valid_loss, 99) - statistics.stdev(valid_loss) 

print(threshold_1)
print(threshold_2)
print(threshold_3)

0.19576008091809938
0.33034461736679077
0.20584060850341562


## 모델 테스트

* 이전 단계에서 계산한 임계값을 기준으로 테스트 데이터를 통한 모델 테스트 진행

In [21]:
# test 데이터 모델 적용 
test_loss = []
with torch.no_grad():
    
    for batch_idx, test_data in enumerate(test_dataloader):

        inputs_test = test_data.to(device).float()

        outputs_test = model_(inputs_test)
        loss = criterion(outputs_test, inputs_test)
        
        test_loss.append(loss.item())
        

# 테스트 결과 데이터 프레임 생성
result = pd.DataFrame(test_loss, columns=['Reconst_Loss'])
# 비정상 데이터는 없다고 가정 
result['label'] = 0

# 각 임계값 기준 정상, 비정상 구분
result['pred_1'] = np.where(result['Reconst_Loss']>threshold_1,1,0)
result['pred_2'] = np.where(result['Reconst_Loss']>threshold_2,1,0)
result['pred_3'] = np.where(result['Reconst_Loss']>threshold_3,1,0)


# 성능 평가

* F1 Score 기준으로 평가 진행 
* 임계값 별로 성능 평가 진행 후 가장 좋은 성능을 보인 임계값을 fix

In [22]:
# 1. 평균 + 편차 임계값 설정 
print(classification_report(result['label'], result['pred_1'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      0.84      0.91       113

   micro avg       1.00      0.84      0.91       113
   macro avg       1.00      0.84      0.91       113
weighted avg       1.00      0.84      0.91       113



In [23]:
# 2. Max 임계값 설정
print(classification_report(result['label'], result['pred_2'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       113

    accuracy                           1.00       113
   macro avg       1.00      1.00      1.00       113
weighted avg       1.00      1.00      1.00       113



In [24]:
# 3. 99% - std 임계값 설정 
print(classification_report(result['label'], result['pred_3'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      0.86      0.92       113

   micro avg       1.00      0.86      0.92       113
   macro avg       1.00      0.86      0.92       113
weighted avg       1.00      0.86      0.92       113

